# Исследование надежности заемщиков

В данном проекте мы исследуем статистику о платежеспособности клиентов банка для того чтобы посмотреть, есть ли зависимость между семейным положением, количеством детей или уровнем достатка и процентом задолжностей по кредитам. 

Данное исследование поможет в даньнейшем построении построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. 

План исследования состоит из двух этапов: 
- Предобработка данных 
- Анализ зависимостей

В первом этапе проекта будет проведен анализ данных, удаление пропусков и дубликатов, изменение типов данных и обработка аномальных значений, а также категоризация данных. 

Во втором этапе проекта мы рассмотрим наличие зависимостей между данными и сделаем общий вывод о показателях, которые влияют на наличие долгов по кредитам. 

Кроме того в итоговом выводе мы постараемся сформировать портрет идеального заёмщика для банка.

### 1. Обзор данных

In [1]:
import pandas as pd # импортируйте библиотеку pandas

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('data.csv') # прочитаем csv-файл               

In [3]:
print(data.head(10)) 

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. Предобработка данных

### Удаление пропусков

In [5]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [6]:
for i in data['income_type'].unique():
    data.loc[(data['income_type'] == i) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == i), 'total_income'].median()
print(data['total_income'].isna().sum())

0


### Обработка аномальных значений

In [7]:
data['days_employed'] = abs(data['days_employed'])

In [8]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
data = data[(data['children'] != -1 ) & (data['children'] != 20 )]
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
for d in data['income_type'].unique():
    data.loc[(data['income_type'] == d) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == d), 'days_employed'].median()
data.isna().sum()    

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [12]:
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

In [13]:
data['education'] = data['education'].str.lower()

In [14]:
data.duplicated().sum()

71

In [15]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.



In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [17]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [18]:
def categorize_purpose(row):
    try:
        if  'автом' in row:
            return 'операции с автомобилем'
        elif 'недвиж' in row or 'жил' in row:
            return 'операции с недвижимостью'
        elif 'свадьб' in row:
            return 'проведение свадьбы'
        elif 'образ' in row:
            return 'получение образования'
    except:
        return 'нет категории'
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для того, чтобы исследовать есть ли зависимость между количеством детей и возвратом кредита в срок, нужно узнать какое количество должников в каждой группе людей с различным количеством детей. Для начала узнаем сколько детей может быть в семьях. 

In [19]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Далее посмотрим количество кредитов и долгов по семьям с различным количеством детей.

In [20]:
data_children_debt = data.groupby('children').agg({'debt': ['count', 'sum']}) 
data_children_debt

debt      
          count   sum
children             
0         14091  1063
1          4808   444
2          2052   194
3           330    27
4            41     4
5             9     0

В столбце count мы видим общее количество кредитов выданное клиентам банка в зависимости от количества детей. В слолбце sum - количество клиентов с задолжностью по кредитам в зависимости от количества детей.
Добавим столбец который расчитает процент долгов от общего количества кредитов.

In [21]:
data_children_debt['pursentage_debt'] = data_children_debt['debt']['sum']/data_children_debt['debt']['count']*100
data_children_debt

debt       pursentage_debt
          count   sum                
children                             
0         14091  1063        7.543822
1          4808   444        9.234609
2          2052   194        9.454191
3           330    27        8.181818
4            41     4        9.756098
5             9     0        0.000000

Исходя из таблицы, мы можем заметить, что количество людей, имющих 4 и 5 детей невилеко (меньше 100). По моему мнению данные группы семей имеют недостаточно количество информции, чтобы их включать в анализ и сравнивать с другими группами, по которым данных значительно больше. Поэтому уберём из нашей статистики семьи с 4 и 5 детьми.

In [22]:
data_children_debt = data_children_debt.loc[data_children_debt['debt']['count'] > 100].sort_values(by='pursentage_debt',ascending=True)
data_children_debt

debt       pursentage_debt
          count   sum                
children                             
0         14091  1063        7.543822
3           330    27        8.181818
1          4808   444        9.234609
2          2052   194        9.454191

Вывод: Семьи которые имеют детей чаще опаздывают с выплатами по кредитам поэтому имеют больший процент долгов, чем семьи без детей. Возможно это связанно с большим количеством незапланированных расходов на детей.

Кроме того семьи с тремя детьми имеют меньший процент задолжностей, чем остальные семьи с детьми. Возможно это связанно с более взрослыми родителями, которые берут кредит, так как семьи с 3мя детьми формируются дольше чем семьи с 1 и 2 детьми. Кроме того у людей с 3мя детьми возможно появляется больше ответственности и они не хотят брать кредиты, а если и берут, то более стабильно их выплачивают.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
#Посмотрим распределение кредитов и долгов в зависимости от семейного положения с помощью группировки столбца family_status.
data_family_status_debt = data.groupby('family_status').agg({'debt': ['count', 'sum']}) 
#Добавим столбец который расчитает процент долгов от общего количества кредитов. 
data_family_status_debt['pursentage_debt'] = data_family_status_debt['debt']['sum']/data_family_status_debt['debt']['count']*100
#Отсортируем таблицу по возрастанию показателя pursentage_debt
data_family_status_debt.sort_values(by='pursentage_debt',ascending=True)

debt      pursentage_debt
                       count  sum                
family_status                                    
вдовец / вдова           951   63        6.624606
в разводе               1189   84        7.064760
женат / замужем        12261  927        7.560558
гражданский брак        4134  385        9.313014
Не женат / не замужем   2796  273        9.763948

Вывод: Наименьшее количество долгов имеют вдовцы и люди в разводе. Возможно это связано с возрастов клиентов, которые имеют такое сеемейное положение. Ведь большенство людей разводятся или становяться вдовцами в более зрелом возрасте. Клиенты старшего возраста более ответсвенно относятся к выплате кредитов нежели молодые люди.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
#Посмотрим распределение кредитов и долгов в зависимости от уровня доходов с помощью группировки столбца total_income_category.
data_income_category_debt = data.groupby('total_income_category').agg({'debt': ['count', 'sum']}) 
#Добавим столбец который расчитает процент долгов от общего количества кредитов. 
data_income_category_debt['pursentage_debt'] = data_income_category_debt['debt']['sum']/data_income_category_debt['debt']['count']*100
data_income_category_debt

debt       pursentage_debt
                       count   sum                
total_income_category                             
A                         25     2        8.000000
B                       5014   354        7.060231
C                      15921  1353        8.498210
D                        349    21        6.017192
E                         22     2        9.090909

In [25]:
#Исключим из таблиц50ы группы в которых меннее 100 человек
data_income_category_debt = data_income_category_debt.loc[data_income_category_debt['debt']['count'] > 100]
#Отсортируем таблицу по возрастанию показателя pursentage_debt
data_income_category_debt.sort_values(by='pursentage_debt',ascending=True)

debt       pursentage_debt
                       count   sum                
total_income_category                             
D                        349    21        6.017192
B                       5014   354        7.060231
C                      15921  1353        8.498210

Вывод: Наиболее добросовестными клиентами, которые имеют меньше всего задолжностей, являются люди с достатком от 30 001 до 50 000(категория D).

In [26]:
#Зададим названия для трёх видов дохода
income_type = ['низкий доход', 'средний доход', 'высокий доход']
#Создадим 3 новых равномерных интервала из столбца total_income
data['income_type'] = pd.qcut(data['total_income'], q=3, precision=0, labels=income_type)
data['income_type'].value_counts()

средний доход    7238
низкий доход     7111
высокий доход    6982
Name: income_type, dtype: int64

Вывод: Клиенты с высоким доходом более ответсвенно подходят к выплатам кредитов, процент долга у людей с высоким доходом составляет 7,5%. Люди со средним доходом более подвержены накапливать долги по кредитам. Количество клиентов со средним достатком, у которых есть долги составляет 8,6%.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [27]:
#Посмотрим распределение кредитов и долгов в зависимости от цели кредита с помощью группировки столбца purpose_category.
data_purpose_category_debt = data.groupby('purpose_category').agg({'debt': ['count', 'sum']}) 
#Добавим столбец который расчитает процент долгов от общего количества кредитов. 
data_purpose_category_debt['pursentage_debt'] = data_purpose_category_debt['debt']['sum']/data_purpose_category_debt['debt']['count']*100
#Отсортируем таблицу по возрастанию показателя pursentage_debt
data_purpose_category_debt.sort_values(by='pursentage_debt',ascending=True)

debt      pursentage_debt
                          count  sum                
purpose_category                                    
операции с недвижимостью  10751  780        7.255139
проведение свадьбы         2313  183        7.911803
получение образования      3988  369        9.252758
операции с автомобилем     4279  400        9.347978

Вывод: Кредиты на недвижимость наиболее распространённая причина займа. 
Люди которые берут кредит на образование или покупку автомобиля платят кредит менее исправно, чем люди берущие кредит на покупку жилья или проведение свадьбы.  Это может быть связано с тем, что приобритение жилья более серьёзная и дорогая покупка, выплаты по которой обычно растягиваются на несколько десятков лет, поэтому ежемесячный платёж небольшой. А на свадьбу наоборот берут самый маленький кредит который легко выплачивать. 

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

В исходных данных пропуски были в двух столбцах: total_income(сумма дохода) и days_employed(стаж работы в днях). Количество пропусков в данных столбцах одинаково, поэтому, можно предположить, что проблемма технического характера. Возможно анкета со временем изменилась и добавились дополнительные столбцы для заполнения, а файл с которым мы работали содержит и старую версию анкеты и новую версию, поэтому количество пропусков одинакого.
Кроме того, причина может быть неслучайной. Некоторые люди скрывают данные о своём достатке или трудовом стаже. Обычно данные люди находятся на высоких долностях и имеют высшее образование.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Медианное значение будет более справедливым, чем другие, так как при таком заполнении не теряются показатели, соответственно выборка будет более полной. Кроме того, в сревнении со средним значение, медиана не искажается очень низкими или очень высокими параметрами из выборки для определённой группы. 

## 4. Общий вывод.

Проект состоит из двух частей. В первой части нашего проекта мы сделали предобработку данных. Датафрейм состоит из 12 колонок с  информацией о клиентах банка:
- количество детей
- стаж работы в днях 
- возраст клиента в годах
- уровень образования клиента
- идентификатор уровня образования
- семейное положение
- идентификатор семейного положения
- пол клиента
- тип занятости
- имел ли задолженность по возврату кредитов
- ежемесячный доход
- цель получения кредита

В ходе просмотра данных обнаружелись пропуски в двух колонках (стаж работы в днях и ежемесячный доход). 
Пропуски в солбце total_income мы заменили на медианное значиение по каждому типу из столбца income_type. 
Пропуски в столбце days_employed мы заполнили медианными значениями по каждому типу занятости income_type.

Кроме того были обработаны аномальные значения в столбцах days_employed и children. 
Также в столбце с информацией по ежемесячному доходу мы заменили вещественный тип данных на целочисленный с помощью метода astype().

В солбце education были исключены все дубликаты, путём использования метода drop_duplicates().

В конце концов была проведена категоризация данных по ежемесячному доходу и цели получения кредита. 
Данные по ежемесячному доходу разбили на 5 диапазонов со следующими категориями: 

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Данные по цели получения кредита разбили на 4 диапазона с категориями: 

- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования' #


Во второй части проекта мы провели анализ влияния некоторых параметров на количество долгов. 
Исходя из проведённого исследования мы можем сделать вывод, что существует несколько параметров, которые влияют на стабильные выплаты по кредитам. 

- Наличие детей  

Семьи, которые имеют детей, наиболее вероятно будут задерживать выплаты по кредитам. Процент задержки выплат по кредиту у семей с детьми составляет от 8,1% до 9,5%, для сравнения доля клиентов без детей, которые имеют задолжности составляет 7,5%. 

- Семейное положение

Семейное положение также имеет влияние на добросовестное выплачивание кредита. Судя по статистике наименьшее количество задолжностей имеют люди потерявшие своего партнёра из-за развода или смерти. Такие клиенты не имею поддержки супруга, поэтому более ответственно относятся к выплачиванию кредита.  Люди которые находятся в гражданском браке или в официальном находятся посередине выборки. Наибольшее количество задолжностей наблюдается у людей, которые никогда не состояли в браке.

- Уровень дохода

#Клиенты, зарабатывающие от 30 001 до 50 000 меньше всего имеют долгов по кредитам, всего лишь 6%. Для сравнения у клиентов с достатном от 50 001 до 200 000 процент долга составляет 8,5%. 

Клиенты с высоким доходом более вероятно будут стабильно выплачивать кредит добросовестно, так как располагают большим количеством денег, поэтому не нужно думать о том на какие деньги им придется жить после выплат по кредитам.Долгов у данной категории людей меньше всего, лишь 7,5%. Процент людей с низким достатком и долгами по кредитам составляет 8,1%. Количество клиентов со средним достатком, у которых есть долги составляет 8,6%. 

- Цель кредита

Цель кредита также влияет на количество долгов по кредитам. Люди занимавшие деньги для свадьбы или для покупки недвижимости более добросовестно выплачивают кредиты. Процент долгов составляет 7,9% и 7,2% соответственно. В категории людей бравших кредит для образования или покупки машины мы видим увеличение количества долгов. 9,2% клиентов взявших кредит для получения образования задерживают по нему выплаты. Аналогичная ситуация и с кредитами для покупки авто, 9,3%. 

Таким образом, можно сделать вывод, что идеальный клиент для банка будет гражданин без детей в разводе или вдовец, с высоким достатком, который берёт кредит для покупки недвижимости. При взаимодействии с таким клиентом у банка будет наименьшее количество проблем, а вероятность добросовестной оплаты по кредиту возрастёт. 

В дальнейшем необходимо провести исследование, которое определит есть ли взаимосвязь между возрастом и количеством долгов по кредиту, а также между наличием образования и процентом долгов. Кроме того, в будущем, возможно будет более объёмная информация о клиентах и мы сможем посмотреть на другие параметры, которые также могут влиять на процент долгов по кредиту, например, наличие недвижимости. 